In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

examples=[
    {
        "영화":"파묘",
        "answer":"""
        상세정보:
        감독:장재현
        각본:장재현
        출연 : 최민식,김고은,유해진
        장르: 호러 영화,스릴러 영화,미스터리
        영화 특징: 으스스한, 불길한, 어두운
        상영시간:02시간 14분
        """,
    },{
        "영화": "브로큰",
        "answer": """
        상세정보:
        감독: 김진황
        각본: 김진황
        출연: 하정우, 김남길, 유다인, 정만식
        장르: 스릴러, 드라마
        영화 특징: 긴장감 넘치는, 감정적인, 어두운
        상영시간: 2시간 5분
        """
    },{
        "영화": "히트맨 2",
        "answer": """
        상세정보:
        감독: 최원섭
        각본: 최원섭, 신중렬
        출연: 권상우, 정준호, 이이경, 황우슬혜, 김성오
        장르: 액션, 코미디
        영화 특징: 유머러스한, 빠른 전개, 경쾌한
        상영시간: 1시간 58분
        """
    },
]

example_prompt=ChatPromptTemplate.from_messages(
    [
        ("human","{영화}에 대해 알고있는 것은?"),
        ("ai","{answer}")
    ]
)

example_prompt=FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt=ChatPromptTemplate.from_messages(
    [
        ("system","넌 영화 전문가야, 한국말로 예제 처럼 대답해줘"),
        example_prompt,
        ("human","{영화}에 대해 알고있는 것은?"),
    ]
)

chain=final_prompt|chat
chain.invoke({"영화":"겨울왕국"})


        상세정보:
        감독: 크리스 벅, 제니퍼 리
        각본: 크리스 벅, 제니퍼 리, 셰인드라 블레이크
        출연: 크리스틴 벨, 이디나 멘젤, 조시 게드, 조나단 그로프
        장르: 애니메이션, 뮤지컬, 모험
        영화 특징: 따뜻한, 감동적인, 음악적
        상영시간: 1시간 42분
        

AIMessageChunk(content='\n        상세정보:\n        감독: 크리스 벅, 제니퍼 리\n        각본: 크리스 벅, 제니퍼 리, 셰인드라 블레이크\n        출연: 크리스틴 벨, 이디나 멘젤, 조시 게드, 조나단 그로프\n        장르: 애니메이션, 뮤지컬, 모험\n        영화 특징: 따뜻한, 감동적인, 음악적\n        상영시간: 1시간 42분\n        ')